In [48]:
### LOCATION DATA SCRIPTS ###
import os
import sys
import csv
import argparse
import pandas as pd
import time

# https://simplemaps.com/data/us-cities

# twitter.com/anyuser/status/541278904204668929

hashtag_state_full =  ['\'Alabama\'', '\'Alaska\'', '\'Arizona\'', '\'Arkansas\'', \
'\'California\'', '\'Colorado\'', '\'Connecticut\'', '\'Delaware\'', '\'Florida\'', \
'\'Georgia\'', '\'Hawaii\'', '\'Idaho\'', '\'Illinois\'' , '\'Indiana\'' , '\'Iowa' , \
'\'Kansas\'' , '\'Kentucky\'' , '\'Louisiana\'' , '\'Maine\'' , '\'Maryland\'' , \
'\'Massachusetts\'' , '\'Michigan\'' , '\'Minnesota\'' , '\'Mississippi\'' , \
'\'Missouri\'' , '\'Montana\'' '\'Nebraska\'' , '\'Nevada' , '\'NewHampshire\'' , \
'\'NewJersey\'' , '\'NewMexico\'' , '\'NewYork\'' , '\'NorthCarolina\'' , \
'\'NorthDakota\'' , '\'Ohio' , '\'Oklahoma\'' , '\'Oregon\'' , '\'Pennsylvania\'', \
'\'RhodeIsland\'', '\'SouthCarolina\'' , '\'SouthDakota\'' , '\'Tennessee\'' , \
'\'Texas\'' , '\'Utah\'' , '\'Vermont\'' , '\'Virginia\'' , '\'Washington\'' , \
'\'WestVirginia\'' , '\'Wisconsin\'', '\'Wyoming\'']

hashtag_state_abbr = ['\'AL\'', '\'AK\'', '\'AZ\'', '\'AR\'', '\'CA\'', '\'CO\'', \
    '\'CT\'', '\'DE\'', '\'FL\'', '\'GA\'', '\'HI\'', '\'ID\'', '\'IL\'', '\'IN\'', \
    '\'IA\'', '\'KS\'', '\'KY\'', '\'LA\'', '\'ME\'', '\'MD\'', '\'MA\'', '\'MI\'', \
    '\'MN\'', '\'MS\'', '\'MO\'', '\'MT\'', '\'NE\'', '\'NV\'', '\'NH\'', '\'NJ\'', \
    '\'NM\'', '\'NY\'', '\'NC\'', '\'ND\'', '\'OH\'', '\'OK\'', '\'OR\'', '\'PA\'', \
    '\'RI\'', '\'SC\'', '\'SD\'', '\'TN\'', '\'TX\'', '\'UT\'', '\'VT\'', '\'VA\'', \
    '\'WA\'', '\'WV\'', '\'WI\'', '\'WY\'']



city_state_abbr = []
city_state_full = []
city_list_indicies = []

city_state_abbr_list = {}


location_list = list(zip(hashtag_state_full, hashtag_state_abbr))

for pair in location_list:
    city_state_abbr_list[pair[1]] = pair[1] 

def make_domain_csv(domain_dict):
    with open('top_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])
            
def city_master_list():
    
    df = pd.read_csv('/Users/tdt62/Desktop/GraduateResearch/scripts/location_data/uscitiesv1.4.csv', index_col=None, header=0)

    return df


def find_city_state(TSV_file, TSV_output_dir, df):
    
    city_state_dict = {}
    lat_lon_dict = {}
    
    lat_lon_list = zip(df['city'], zip(df['lat'], df['lng']))
    city_state = list(zip(df['city'], df['state_name']))
    
    for pair in lat_lon_list:
        lat_lon_dict[pair[0].upper()] = pair[1]
    
    for pair in city_state:
        if pair[1].upper() in city_state_dict:
            city_state_dict[pair[1].upper()].append((pair[0].upper()))
        else:
            city_state_dict[pair[1].upper()] = [(pair[0].upper())]
    
    twitter_df = pd.read_csv(TSV_file, index_col=None, delimiter='\t')    
    
    def get_location(df):
        locations = []

        cur_tweet = str(df['Full_Text']).upper().split(' ')
        for word in cur_tweet:
            for state, city in city_state_dict.items():
                try:
                    if('http' not in word and 'https' not in word):
                        if word[0] == '#':
                            if word[1:2] in hashtag_state_abbr:
                                locations.append(city_state_abbr_list[word[1:2]])

                            if word[1:] in city:
                                for city_words in city:
                                    if word[1:] in city_words:
                                        locations.append(((city_words, state), (lat_lon_dict[city_words])))

                        else:
                            if word in city:
                                for city_words in city:
                                    if word in city_words:
                                        locations.append(((city_words, state), (lat_lon_dict[city_words])))
                            if word == state:
                                locations.append(state)
                except:
                    continue

        return locations
    try:
        twitter_df = pd.read_csv(TSV_file, index_col=None, delimiter='\t')
        twitter_df['Locations'] = twitter_df.apply(get_location, axis=1)
        twitter_df = twitter_df.drop(['Has_Location', 'Places'], axis=1)
        twitter_df.to_csv(TSV_output_dir + TSV_file.split('/')[-1].split('.')[0] + '_location.csv', sep='\t')
    except TypeError:
        print('TypeError')
        print("Filename: {}".format(TSV_file))
    except KeyError:
        twitter_df = pd.read_csv(TSV_file, index_col=None, delimiter='\t', header=None, names=["Unnamed: 0", "TweetID", "Timestamp", "Full_Text", "In_Reply_To_User_ID", "User_ID", "User_Name", "User_Screen_Name", "Coordinates", "Place", "Bounding_Box", "Quoted_Status_ID", "Retweeted_Status", "Hashtags", "URLs", "User_Mentions", "Media,Language"])
        twitter_df['Locations'] = twitter_df.apply(get_location, axis=1)
#         twitter_df = twitter_df.drop(['Has_Location', 'Places'], axis=1)
        twitter_df.to_csv(TSV_output_dir + TSV_file.split('/')[-1].split('.')[0] + '_location.csv', sep='\t')
    return twitter_df

    
master_list = city_master_list()
frame = find_city_state("/Users/tdt62/Desktop/2019_02_05_19_vote_stream_1.csv", "/Users/tdt62/Desktop/test_data/", master_list)
frame['Locations']


# def parse_args():
#     '''
#     Parses the command line arguments for the script

#     Returns
#     -------

#     args : object
#            Python arg parser object
#     '''
#     parser = argparse.ArgumentParser(description='Process some integers.')

#     parser.add_argument('TSV_file', type=str, help='Twitter stream TSV file to find city states.')

#     parser.add_argument('TSV_output_file', type=str, help='Output file with found city states.')

#     args = parser.parse_args()
    
#     return args

# def main():
#     args = parse_args()

#     city_list_indicies = city_master_list()

#     find_city_state(args.TSV_file, args.TSV_output_file, city_list_indicies)

# if __name__ == "__main__":
#     main()


0      []
1      []
2      []
3      []
4      []
5      []
6      []
7      []
8      []
9      []
10     []
11     []
12     []
13     []
14     []
15     []
16     []
17     []
18     []
19     []
20     []
21     []
22     []
23     []
24     []
25     []
26     []
27     []
28     []
29     []
       ..
180    []
181    []
182    []
183    []
184    []
185    []
186    []
187    []
188    []
189    []
190    []
191    []
192    []
193    []
194    []
195    []
196    []
197    []
198    []
199    []
200    []
201    []
202    []
203    []
204    []
205    []
206    []
207    []
208    []
209    []
Name: Locations, Length: 210, dtype: object

In [2]:
for x in frame['Locations']:
    print(len(x))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [3]:
domain_df = pd.read_csv("/Users/tdt62/Desktop/GraduateResearch/scripts/location_data/projection_csv/jaccard_domain.csv", names=['User_Domain', 'JS'])


In [4]:
def check_website(x):
    if('.co' in x.split(',')[0] and '.co' in x.split(',')[1]):
        return True
    return False

domain_df['website'] = list(map(lambda x: check_website(x), domain_df['User_Domain']))


In [ ]:
new_df = domain_df.loc[domain_df['website'] == True] 
new_df = new_df.loc[domain_df['JS'] != 0.0] 
new_df[['User_Domain', 'JS']].to_csv('edgelist_js.csv')
new_df

,User_Domain,JS,website
11,"('teespring.com', 'www.teenvogue.com')",0.533333,True
16,"('usauknews.com', 'www.houstonchronicle.com')",0.562500,True
18,"('www.redding.com', 'www.newnownext.com')",0.538462,True
19,"(""dorsetecho.co.uk']"", 'www.wibw.com')",0.176471,True
22,"('dailywn.com', 'www.theyeshivaworld.com')",0.588235,True
32,"('cjonline.com', 'm.facebook.com')",0.384615,True
35,"('gogetfunding.com', 'www.consumeraffairs.com')",0.562500,True
49,"('aplus.com', 'homerenovation989.wordpress.com')",0.350000,True
53,"('byellowtail.com', 'www.elle.com')",0.583333,True
64,"('www.instagram.com', 'www.eluniversal.com')",0.625000,True


In [38]:
hash_df = pd.read_csv("/Users/tdt62/Desktop/2018_10_20_14_vote_stream_1_location.csv")
hash_df = hash_df.drop(['Places', 'Has_Location'], axis=1)
hash_df


,Unnamed: 0,Unnamed: 0.1,TweetID,Timestamp,Full_Text,In_Reply_To_User_ID,User_ID,User_Name,User_Screen_Name,Coordinates,Place,Bounding_Box,Quoted_Status_ID,Retweeted_Status,Hashtags,URLs,User_Mentions,Media,Language,Locations
0,0,0,1053752891024932864,2018-10-20 21:00:41,b'Make sure your #FreedomIsFree. There is no P...,NaN,83289253,Mike Van Dyke,ZaKapu,NaN,NaN,[],0,1053717195832799232,"[{'text': 'FreedomIsFree', 'indices': [31, 45]...",[],"[{'screen_name': 'YrachetaJM', 'name': 'Joe M....",[],en,"[(('PRICE', 'MARYLAND'), (39.603999999999999, ..."
1,1,1,1053752905017245696,2018-10-20 21:00:44,b'There are so many amazing candidates this ye...,NaN,14139469,Tim Brown,tmbrown327,NaN,NaN,[],0,1053030524942475264,[],[],"[{'screen_name': 'mattmfm', 'name': 'Matt McDe...",[],en,"[(('MANY', 'LOUISIANA'), (31.566299999999998, ..."
2,2,2,1053753169325498368,2018-10-20 21:01:48,b'Date idea: go to your local election office ...,NaN,2169042721,🎃🖤,Mon_chita69,NaN,"Kansas, USA","[[[-102.051769, 36.99311], [-102.051769, 40.00...",0,[],[],[],[],[],en,[]
3,3,3,1053753190032633856,2018-10-20 21:01:52,b'',NaN,181508223,ǝsǝuɐɹɓ ǝʞıɯ©,JojoPapa32,NaN,NaN,[],0,1053721071646474240,"[{'text': 'ElectWomen', 'indices': [71, 82]}, ...",[],"[{'screen_name': 'Deb4CongressNM', 'name': 'De...",[],en,[]
4,4,4,1053753191827992577,2018-10-20 21:01:53,"b'They regrouped, got JD, Bryan Michael Cox *a...",NaN,78814784,True Lies,TrueLies6,NaN,NaN,[],0,1053752686397546498,[],[],"[{'screen_name': 'naima', 'name': 'Naima Cochr...",[],en,"[(('BRYAN', 'OHIO'), (41.4711, -84.54890000000..."
5,5,5,1053753225264947201,2018-10-20 21:02:01,b'#Deb4CongressNM #VOTEthemOUT\xf0\x9f\x87\xba...,NaN,2761409745,Joe Allen Henry,ItsMe_JoeAllen,NaN,NaN,[],1053450727664697345,[],"[{'text': 'Deb4CongressNM', 'indices': [0, 15]...","[{'url': 'https://t.co/xrN9SWEX8t', 'expanded_...",[],[],en,[]
6,6,6,1053753296500838401,2018-10-20 21:02:18,"b'Mohammed bin Salman met with Oprah, the Clin...",NaN,212629196,Jim Earl,jimearl666,NaN,NaN,[],0,[],[],"[{'url': 'https://t.co/3GfvsUVame', 'expanded_...",[],[],en,"[(('MORGAN', 'MISSOURI'), (38.602800000000002,..."
7,7,7,1053753384891609088,2018-10-20 21:02:39,b'BREAKING: Native American Lesbian Lawyer Sha...,NaN,168062442,KOREA BYS 배용석,SmartFoodDM,NaN,NaN,[],0,1053085368608833538,[],[],"[{'screen_name': 'DisavowTrump20', 'name': 'Pr...",[],en,"[(('LEAD', 'SOUTH DAKOTA'), (44.35269999999999..."
8,8,8,1053753389174018048,2018-10-20 21:02:40,b'\xf0\x9f\x9a\xa8Lets #MAGA For @NatlGovsAsso...,NaN,889522107058577408,❌Veteran Mike❌ {⭐️}(R) #MAGAVet🇺🇸👊🏼🇺🇸,mtenorio77,NaN,NaN,[],0,1052334605125193728,"[{'text': 'MAGA', 'indices': [28, 33]}]",[],"[{'screen_name': 'DeplorableJuJuV', 'name': '🇺...",[],en,[]
9,9,9,1053753437584846849,2018-10-20 21:02:51,b'Pet sitters are the unsung heroes of elector...,NaN,15059667,Peggy Flanagan (Untamed),peggyflanagan,NaN,NaN,[],0,[],[],[],[],"[{'id': 1053753431016361984, 'id_str': '105375...",en,[]


In [41]:
twitter_df = pd.read_csv("/Users/tdt62/Desktop/2019_02_05_19_vote_stream_1.csv", index_col=None, delimiter='\t', header=None, names=["Unnamed: 0", "TweetID", "Timestamp", "Full_Text", "In_Reply_To_User_ID", "User_ID", "User_Name", "User_Screen_Name", "Coordinates", "Place", "Bounding_Box", "Quoted_Status_ID", "Retweeted_Status", "Hashtags", "URLs", "User_Mentions", "Media,Language"])
twitter_df


,Unnamed: 0,TweetID,Timestamp,Full_Text,In_Reply_To_User_ID,User_ID,User_Name,User_Screen_Name,Coordinates,Place,Bounding_Box,Quoted_Status_ID,Retweeted_Status,Hashtags,URLs,User_Mentions,"Media,Language"
0,1092966733470564352,2019-02-06 02:02:30,b'@realT_RumpT_Rex @maiamimi @UnionStationKC @...,9.326576e+17,925524815414480896,Sonny Scroggins,sonny_scroggins,NaN,NaN,[],0,[],[],['https://twitter.com/i/web/status/10929667334...,"{'realT_RumpT_Rex': 932657569952301056, 'maiam...",[],und
1,1092966762394472448,2019-02-06 02:02:37,b'@realT_RumpT_Rex @maiamimi @UnionStationKC @...,NaN,925524815414480896,Sonny Scroggins,sonny_scroggins,NaN,NaN,[],0,1092966733470564352,[],[],"{'sonny_scroggins': 925524815414480896, 'realT...",[],und
2,1092966858972553216,2019-02-06 02:03:00,"b'Again this year as he did last time, GOP Rep...",2.369949e+09,2369949012,Joe Perticone,JoePerticone,NaN,NaN,[],0,[],[],[],{},[],en
3,1092966891465859072,2019-02-06 02:03:07,"b'Qu\xc3\xa9 Alba de porquer\xc3\xada, no tien...",NaN,976891949977358336,mr. brightside💔,Ntr266,NaN,NaN,[],0,[],[],['https://twitter.com/i/web/status/10929668914...,{},[],es
4,1092966898810015745,2019-02-06 02:03:09,b'',NaN,218325695,Ben Jacobs,Bencjacobs,NaN,NaN,[],0,1092966858972553216,[],[],{'JoePerticone': 2369949012},[],en
5,1092966920087695365,2019-02-06 02:03:14,b'Meet @realDonaldTrump &amp; @FLOTUS\xe2\x80\...,NaN,18490056,Garren Shipley,GarrenShipley,NaN,NaN,[],0,1092947229999054848,['SOTU'],[],"{'GOP': 11134252, 'realDonaldTrump': 25073877,...",[],en
6,1092966965432307712,2019-02-06 02:03:25,"b'Beiruts Green Line demarcation zone, which s...",NaN,70834865,ObliviousRydian,ObliviousRydian,NaN,NaN,[],0,1092756559179014144,[],[],{'UrbanFoxxxx': 3404085119},[],en
7,1092967104469331968,2019-02-06 02:03:58,"b'Qu\xc3\xa9 Alba de porquer\xc3\xada, no tien...",NaN,976891949977358336,mr. brightside💔,Ntr266,NaN,NaN,[],0,[],[],['https://twitter.com/i/web/status/10929671044...,{},[],es
8,1092967144071999488,2019-02-06 02:04:08,b'Among the president\xe2\x80\x99s guests for ...,NaN,21711202,Arif Rafiq,ArifCRafiq,NaN,NaN,[],0,[],[],['https://twitter.com/i/web/status/10929671440...,{},[],en
9,1092967302482452483,2019-02-06 02:04:45,b'Meet @realDonaldTrump &amp; @FLOTUS\xe2\x80\...,NaN,850447999352352769,GOP News,gop_rules,NaN,NaN,[],0,1092947229999054848,['SOTU'],[],"{'GOP': 11134252, 'realDonaldTrump': 25073877,...",[],en
